In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter
import mlflow
import sklearn
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 

# local imports
from prepare import *
from evaluate import *

Using TensorFlow backend.
C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\tensorflow\python\util\nest.py:1286: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  _pywrap_tensorflow.RegisterType("Mapping", _collections.Mapping)
C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\604

In [2]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from catboost import CatBoostRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import (
    LogisticRegression,
    SGDRegressor,
    Ridge,
    LinearRegression,
    Lasso
)
import warnings

warnings.filterwarnings(
    "ignore"
)  # Ridge classifier throws some warnings about ill-conditioned matrix

# Try out balanced classes
reduced_train = pd.read_csv("reduce_train.csv")

# reduced_train = pd.read_csv('reduce_train.csv')
reduced_test = pd.read_csv("reduce_test.csv")
reduced_train.shape, reduced_test.shape

categoricals = ["session_title"]
cols_to_drop = ["game_session", "installation_id", "accuracy_group"]


In [9]:
NN_model = Sequential()


# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal', input_dim=887, activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dropout(0.2))

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dropout(0.2))

NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dropout(0.2))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile model
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


## Setup Models

In [27]:
lgbm = LGBMRegressor()
rfr = RandomForestRegressor()
cbr = CatBoostRegressor(
    loss_function="RMSE",
    task_type="CPU",
    learning_rate=0.05,
    iterations=2000,
    od_type="Iter",
    early_stopping_rounds=500,
    random_seed=42,
    silent=True
)
gbr = GradientBoostingRegressor()
abr = AdaBoostRegressor()
lvr = LinearSVR()
lr = LinearRegression()
sgd = SGDRegressor()
rr = Ridge()

models = [kr, lgbm, rfr, cbr, gbr, abr, lr, rr]

In [28]:
def my_metric(y_true, y_pred):
    y_pred = get_class_pred(y_pred, reduced_train)
    qwk = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    return qwk

### Read in data, set up stack

In [29]:
from vecstack import stacking

mms = MinMaxScaler()

#  Read in and scale data

# If running for submission use these 3 lines:
# X_train = reduced_train.drop(cols_to_drop, axis=1, errors="ignore")
# y_train = reduced_train.accuracy_group
# X_test = reduced_test.drop(cols_to_drop, axis=1, errors="ignore")

# If testing locally, use these line:
X = reduced_train.drop(cols_to_drop, axis=1, errors='ignore')
y = reduced_train.accuracy_group
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_test = mms.fit_transform(X_train), mms.transform(X_test)

In [30]:
S_train, S_test = stacking(
    models,
    X_train,
    y_train,
    X_test,
    regression=True,
    mode="oof_pred_bag",
    needs_proba=False,
    save_dir=None,
    metric=my_metric,
    n_folds=4,
    shuffle=True,
    random_state=42,
    verbose=2,
)

task:         [regression]
metric:       [my_metric]
mode:         [oof_pred_bag]
n_models:     [8]

model  0:     [KerasRegressor]
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_47 (Dense)             (None, 887)               787656    
_________________________________________________________________
dropout_26 (Dropout)         (None, 887)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 256)               227328    
_________________________________________________________________
dropout_27 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_28 (Dropout)         (None, 256)             

Epoch 30/50
 - 0s - loss: 0.5800 - mean_absolute_error: 0.5800
Epoch 31/50
 - 0s - loss: 0.5858 - mean_absolute_error: 0.5858
Epoch 32/50
 - 0s - loss: 0.5750 - mean_absolute_error: 0.5750
Epoch 33/50
 - 0s - loss: 0.5550 - mean_absolute_error: 0.5550
Epoch 34/50
 - 0s - loss: 0.5532 - mean_absolute_error: 0.5532
Epoch 35/50
 - 0s - loss: 0.5568 - mean_absolute_error: 0.5568
Epoch 36/50
 - 0s - loss: 0.5459 - mean_absolute_error: 0.5459
Epoch 37/50
 - 0s - loss: 0.5430 - mean_absolute_error: 0.5430
Epoch 38/50
 - 0s - loss: 0.5312 - mean_absolute_error: 0.5312
Epoch 39/50
 - 0s - loss: 0.5331 - mean_absolute_error: 0.5331
Epoch 40/50
 - 0s - loss: 0.5387 - mean_absolute_error: 0.5387
Epoch 41/50
 - 0s - loss: 0.5183 - mean_absolute_error: 0.5183
Epoch 42/50
 - 0s - loss: 0.5130 - mean_absolute_error: 0.5130
Epoch 43/50
 - 0s - loss: 0.5040 - mean_absolute_error: 0.5040
Epoch 44/50
 - 0s - loss: 0.5144 - mean_absolute_error: 0.5144
Epoch 45/50
 - 0s - loss: 0.5013 - mean_absolute_error:

 - 0s - loss: 0.7771 - mean_absolute_error: 0.7771
Epoch 11/50
 - 0s - loss: 0.7585 - mean_absolute_error: 0.7585
Epoch 12/50
 - 0s - loss: 0.7487 - mean_absolute_error: 0.7487
Epoch 13/50
 - 0s - loss: 0.7445 - mean_absolute_error: 0.7445
Epoch 14/50
 - 0s - loss: 0.7272 - mean_absolute_error: 0.7272
Epoch 15/50
 - 0s - loss: 0.7130 - mean_absolute_error: 0.7130
Epoch 16/50
 - 0s - loss: 0.6979 - mean_absolute_error: 0.6979
Epoch 17/50
 - 0s - loss: 0.6873 - mean_absolute_error: 0.6873
Epoch 18/50
 - 0s - loss: 0.6705 - mean_absolute_error: 0.6705
Epoch 19/50
 - 0s - loss: 0.6576 - mean_absolute_error: 0.6576
Epoch 20/50
 - 0s - loss: 0.6655 - mean_absolute_error: 0.6655
Epoch 21/50
 - 0s - loss: 0.6396 - mean_absolute_error: 0.6396
Epoch 22/50
 - 0s - loss: 0.6271 - mean_absolute_error: 0.6271
Epoch 23/50
 - 0s - loss: 0.6187 - mean_absolute_error: 0.6187
Epoch 24/50
 - 0s - loss: 0.6158 - mean_absolute_error: 0.6158
Epoch 25/50
 - 0s - loss: 0.6059 - mean_absolute_error: 0.6059
Epoc

KeyboardInterrupt: 

In [9]:
model = RandomForestRegressor()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))


Final prediction score: [0.57970312]


In [10]:
model = LinearRegression()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.59833030]


In [11]:
model = LGBMRegressor()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.59167776]


In [12]:
model = Ridge()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.59833030]


In [14]:
model = LinearSVR()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.59563333]


In [15]:
model = AdaBoostRegressor()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.58113770]


In [16]:
model = CatBoostRegressor()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

0:	learn: 1.2446442	total: 5.04ms	remaining: 5.04s
1:	learn: 1.2319451	total: 10ms	remaining: 5s
2:	learn: 1.2198910	total: 14.5ms	remaining: 4.81s
3:	learn: 1.2080398	total: 17.6ms	remaining: 4.38s
4:	learn: 1.1968234	total: 20.8ms	remaining: 4.13s
5:	learn: 1.1862255	total: 24.2ms	remaining: 4.01s
6:	learn: 1.1760127	total: 29.1ms	remaining: 4.12s
7:	learn: 1.1664184	total: 32.9ms	remaining: 4.08s
8:	learn: 1.1574516	total: 36.5ms	remaining: 4.02s
9:	learn: 1.1486729	total: 40.3ms	remaining: 3.99s
10:	learn: 1.1403095	total: 43.7ms	remaining: 3.92s
11:	learn: 1.1324002	total: 47.3ms	remaining: 3.89s
12:	learn: 1.1250868	total: 50.7ms	remaining: 3.85s
13:	learn: 1.1178212	total: 53.8ms	remaining: 3.79s
14:	learn: 1.1110368	total: 57.4ms	remaining: 3.77s
15:	learn: 1.1047284	total: 71.3ms	remaining: 4.38s
16:	learn: 1.0988714	total: 74.7ms	remaining: 4.32s
17:	learn: 1.0931436	total: 78ms	remaining: 4.25s
18:	learn: 1.0875169	total: 81.2ms	remaining: 4.19s
19:	learn: 1.0821300	total: 8

195:	learn: 0.9750778	total: 673ms	remaining: 2.76s
196:	learn: 0.9750101	total: 677ms	remaining: 2.76s
197:	learn: 0.9749154	total: 680ms	remaining: 2.75s
198:	learn: 0.9748426	total: 683ms	remaining: 2.75s
199:	learn: 0.9747902	total: 687ms	remaining: 2.75s
200:	learn: 0.9747324	total: 690ms	remaining: 2.74s
201:	learn: 0.9746792	total: 694ms	remaining: 2.74s
202:	learn: 0.9745858	total: 697ms	remaining: 2.73s
203:	learn: 0.9745363	total: 700ms	remaining: 2.73s
204:	learn: 0.9744937	total: 703ms	remaining: 2.72s
205:	learn: 0.9744469	total: 706ms	remaining: 2.72s
206:	learn: 0.9743370	total: 709ms	remaining: 2.71s
207:	learn: 0.9742319	total: 712ms	remaining: 2.71s
208:	learn: 0.9741755	total: 715ms	remaining: 2.71s
209:	learn: 0.9741080	total: 718ms	remaining: 2.7s
210:	learn: 0.9740300	total: 721ms	remaining: 2.7s
211:	learn: 0.9739100	total: 725ms	remaining: 2.69s
212:	learn: 0.9738387	total: 728ms	remaining: 2.69s
213:	learn: 0.9737801	total: 731ms	remaining: 2.69s
214:	learn: 0.

400:	learn: 0.9592211	total: 1.34s	remaining: 2s
401:	learn: 0.9591174	total: 1.34s	remaining: 2s
402:	learn: 0.9590448	total: 1.35s	remaining: 2s
403:	learn: 0.9589851	total: 1.35s	remaining: 1.99s
404:	learn: 0.9589377	total: 1.35s	remaining: 1.99s
405:	learn: 0.9588971	total: 1.36s	remaining: 1.99s
406:	learn: 0.9588180	total: 1.36s	remaining: 1.98s
407:	learn: 0.9587807	total: 1.36s	remaining: 1.98s
408:	learn: 0.9586897	total: 1.36s	remaining: 1.97s
409:	learn: 0.9586199	total: 1.37s	remaining: 1.97s
410:	learn: 0.9585141	total: 1.37s	remaining: 1.97s
411:	learn: 0.9583574	total: 1.37s	remaining: 1.96s
412:	learn: 0.9582749	total: 1.38s	remaining: 1.96s
413:	learn: 0.9581121	total: 1.38s	remaining: 1.95s
414:	learn: 0.9580259	total: 1.39s	remaining: 1.96s
415:	learn: 0.9579633	total: 1.39s	remaining: 1.96s
416:	learn: 0.9579077	total: 1.4s	remaining: 1.95s
417:	learn: 0.9578076	total: 1.4s	remaining: 1.95s
418:	learn: 0.9577379	total: 1.4s	remaining: 1.94s
419:	learn: 0.9576094	to

605:	learn: 0.9426412	total: 2.02s	remaining: 1.31s
606:	learn: 0.9425429	total: 2.02s	remaining: 1.31s
607:	learn: 0.9424799	total: 2.03s	remaining: 1.31s
608:	learn: 0.9424537	total: 2.03s	remaining: 1.3s
609:	learn: 0.9423829	total: 2.03s	remaining: 1.3s
610:	learn: 0.9422703	total: 2.04s	remaining: 1.3s
611:	learn: 0.9421388	total: 2.04s	remaining: 1.29s
612:	learn: 0.9420534	total: 2.05s	remaining: 1.29s
613:	learn: 0.9420001	total: 2.05s	remaining: 1.29s
614:	learn: 0.9419423	total: 2.05s	remaining: 1.28s
615:	learn: 0.9418657	total: 2.06s	remaining: 1.28s
616:	learn: 0.9417707	total: 2.06s	remaining: 1.28s
617:	learn: 0.9416766	total: 2.06s	remaining: 1.28s
618:	learn: 0.9415545	total: 2.07s	remaining: 1.27s
619:	learn: 0.9414648	total: 2.07s	remaining: 1.27s
620:	learn: 0.9413337	total: 2.08s	remaining: 1.27s
621:	learn: 0.9412670	total: 2.08s	remaining: 1.27s
622:	learn: 0.9412475	total: 2.09s	remaining: 1.26s
623:	learn: 0.9411937	total: 2.1s	remaining: 1.26s
624:	learn: 0.94

790:	learn: 0.9280316	total: 2.69s	remaining: 710ms
791:	learn: 0.9279533	total: 2.69s	remaining: 706ms
792:	learn: 0.9279017	total: 2.69s	remaining: 703ms
793:	learn: 0.9277729	total: 2.7s	remaining: 700ms
794:	learn: 0.9277157	total: 2.7s	remaining: 696ms
795:	learn: 0.9276455	total: 2.7s	remaining: 693ms
796:	learn: 0.9275872	total: 2.71s	remaining: 690ms
797:	learn: 0.9275414	total: 2.71s	remaining: 686ms
798:	learn: 0.9274761	total: 2.71s	remaining: 683ms
799:	learn: 0.9274369	total: 2.72s	remaining: 681ms
800:	learn: 0.9273224	total: 2.73s	remaining: 678ms
801:	learn: 0.9272462	total: 2.73s	remaining: 675ms
802:	learn: 0.9271255	total: 2.74s	remaining: 671ms
803:	learn: 0.9270662	total: 2.74s	remaining: 668ms
804:	learn: 0.9270261	total: 2.74s	remaining: 665ms
805:	learn: 0.9269848	total: 2.75s	remaining: 663ms
806:	learn: 0.9268669	total: 2.76s	remaining: 659ms
807:	learn: 0.9267791	total: 2.76s	remaining: 657ms
808:	learn: 0.9267350	total: 2.77s	remaining: 654ms
809:	learn: 0.9

984:	learn: 0.9145050	total: 3.52s	remaining: 53.7ms
985:	learn: 0.9144619	total: 3.53s	remaining: 50.1ms
986:	learn: 0.9143794	total: 3.53s	remaining: 46.5ms
987:	learn: 0.9143498	total: 3.54s	remaining: 42.9ms
988:	learn: 0.9143179	total: 3.55s	remaining: 39.4ms
989:	learn: 0.9142594	total: 3.55s	remaining: 35.9ms
990:	learn: 0.9141289	total: 3.55s	remaining: 32.3ms
991:	learn: 0.9140925	total: 3.56s	remaining: 28.7ms
992:	learn: 0.9139741	total: 3.56s	remaining: 25.1ms
993:	learn: 0.9139017	total: 3.56s	remaining: 21.5ms
994:	learn: 0.9138427	total: 3.57s	remaining: 17.9ms
995:	learn: 0.9137735	total: 3.57s	remaining: 14.3ms
996:	learn: 0.9137441	total: 3.57s	remaining: 10.8ms
997:	learn: 0.9136748	total: 3.58s	remaining: 7.17ms
998:	learn: 0.9135955	total: 3.58s	remaining: 3.58ms
999:	learn: 0.9135601	total: 3.58s	remaining: 0us
Final prediction score: [0.59311615]


In [22]:
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()
    
reset_keras()

InternalError: GPU sync failed

In [186]:
model = Ridge()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
y_pred = get_class_pred(y_pred, reduced_test)

print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

def make_submission(preds, train_df, filename):
    preds = get_class_pred(preds, train_df)
    # assert len(preds)==1000
    sample = pd.read_csv("data/sample_submission.csv")
    submission = pd.DataFrame()
    submission["installation_id"] = sample["installation_id"]
    submission["accuracy_group"] = preds
    submission.to_csv(filename, index=False)
    return submission

#make_submission(y_pred, reduced_train, 'vs_preds5.csv')

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3
...,...,...
995,fee254cf,3
996,ff57e602,0
997,ffc73fb2,3
998,ffe00ca8,1


In [54]:
make_submission(final_preds, reduced_train)

,installation_id,accuracy_group
0,00abaee7,1
1,01242218,3
2,017c5718,0
3,01a44906,0
4,01bc6cb6,3
...,...,...
995,fee254cf,3
996,ff57e602,1
997,ffc73fb2,3
998,ffe00ca8,3


In [17]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline

In [26]:
def baseline_model():
    NN_model = Sequential()
    
    # The Input Layer :
    NN_model.add(Dense(887, kernel_initializer='normal', input_dim = 887, activation='relu'))
    NN_model.add(Dropout(0.2))

    # The Hidden Layers :
    NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
    NN_model.add(Dropout(0.2))

    NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
    NN_model.add(Dropout(0.2))

    NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
    NN_model.add(Dropout(0.2))

    # The Output Layer :
    NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

    # Compile model
    NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    print(NN_model.summary())
    return NN_model

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_test = mms.fit_transform(X_train), mms.transform(X_test)

# evaluate model with standardized dataset
kr = KerasRegressor(build_fn=baseline_model, epochs=200, batch_size=300, verbose=2)
kr.fit(X_train, y_train)
#results = cross_val_score(pipeline, X, y, cv=kfold)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 887)               787656    
_________________________________________________________________
dropout_22 (Dropout)         (None, 887)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 256)               227328    
_________________________________________________________________
dropout_23 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_24 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 256)             